In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

In [2]:
shape = (224,224,3)
n_class = 1000
alpha = 1
include_top = True

In [3]:
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [4]:
def relu6(x):
    return K.relu(x, max_value=6.0)

def hard_swish(x):
    return x * K.relu(x + 3.0, max_value=6.0) / 6.0

def return_activation(x, nl):
    if nl == 'HS':
        x = layers.Activation(hard_swish)(x)
    if nl == 'RE':
        x = layers.Activation(relu6)(x)

    return x

def conv_block(inputs, filters, kernel, strides, nl):
    x = layers.Conv2D(filters, kernel, strides, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    return return_activation(x, nl)

def SEBlock(inputs,se_ratio = 4):
    filters = int(inputs.shape[3])
    reduced_filters  = int(filters/se_ratio)
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1,1,x.shape[1]))(x)
    x = layers.Conv2D(reduced_filters, 1, 1,use_bias=True)(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, 1, 1,use_bias=True)(x)
    x = layers.Activation('sigmoid')(x)
    x = layers.Multiply()([x, inputs])
    return x

def bottleneck(inputs, filters, kernel, s, squeeze, nl,t):
    input_shape = K.int_shape(inputs)
    output_channel = _make_divisible(filters * alpha, 8)
    exp_size = _make_divisible(input_shape[3] * t, 8)
    r = s == 1 and input_shape[3] == output_channel
    
    x = inputs
    if input_shape[3] != exp_size:
        x = conv_block(x, exp_size, 1, 1, nl)
    x = layers.DepthwiseConv2D(kernel,s, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = return_activation(x, nl)
    if squeeze:
        x = SEBlock(x)
    x = layers.Conv2D(output_channel,1, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if r:
        x = layers.Add()([x, inputs])
    return x

In [9]:
inputs = layers.Input(shape=shape)

x = conv_block(inputs, _make_divisible(16 * alpha, 8), (3, 3), strides=(2, 2), nl='HS')

x = bottleneck(x, 16, 3, s=1, squeeze=False, nl='RE',t=1)
x = bottleneck(x, 24, 3, s=2, squeeze=False, nl='RE',t=4)
x = bottleneck(x, 24, 3, s=1, squeeze=False, nl='RE',t=3)
x = bottleneck(x, 40, 5, s=2, squeeze=True, nl='RE',t=3) 
x = bottleneck(x, 40, 5, s=1, squeeze=True, nl='RE',t=3)
x = bottleneck(x, 40, 5, s=1, squeeze=True, nl='RE',t=3)
x = bottleneck(x, 80, 3, s=2, squeeze=False, nl='HS',t=6)
x = bottleneck(x, 80, 3, s=1, squeeze=False, nl='HS',t=2.5)
x = bottleneck(x, 80, 3, s=1, squeeze=False, nl='HS',t=2.3)
x = bottleneck(x, 80,  3, s=1, squeeze=False, nl='HS',t=2.3)
x = bottleneck(x, 112, 3, s=1, squeeze=True, nl='HS',t=6)
x = bottleneck(x, 112, 3, s=1, squeeze=True, nl='HS',t=6)
x = bottleneck(x, 160, 5, s=2, squeeze=True, nl='HS',t=6)
x = bottleneck(x, 160, 5, s=1, squeeze=True, nl='HS',t=6)
x = bottleneck(x, 160, 5, s=1, squeeze=True, nl='HS',t=6)

x = conv_block(x, _make_divisible(960 * alpha, 8), (1, 1), strides=(1, 1), nl='HS')
x = layers.AveragePooling2D(7)(x)

x = layers.Conv2D(1280, (1, 1), padding='same')(x)
x = return_activation(x, 'HS')

if include_top:
    x = layers.Conv2D(n_class, (1, 1), padding='same', activation='softmax')(x)
    x = layers.Reshape((n_class,))(x)

model = keras.Model(inputs, x)

In [10]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 112, 112, 16) 432         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_92 (BatchNo (None, 112, 112, 16) 64          conv2d_98[0][0]                  
__________________________________________________________________________________________________
activation_80 (Activation)      (None, 112, 112, 16) 0           batch_normalization_92[0][0]     
____________________________________________________________________________________________